## Harmony Py Library

### Basic Workflow Example

This notebook shows three basic examples of Harmony jobs, each using a Harmony test Collection. The first example requests a spatial subset of Alaska, the second a temporal subset (a single-month timespan), and the third shows a combination of both spatial and temporal subsetting.

First, we import a helper module for the notebook, but then import the Harmony Py classes we need to make a request.

In [ ]:
import helper

In [ ]:
import datetime as dt
from harmony import BBox, Client, Collection, Request

First let's prompt for your CMR credentials (UAT). Your credentials are stored without needing to hit enter in either field.

In [ ]:
username = helper.Text(placeholder='captainmarvel', description='Username')
helper.display(username)
password = helper.Password(placeholder='Password', description='Password')
helper.display(password)

Now we create a Harmony Client object, passing in the `auth` tuple containing the username and password entered above.

In [ ]:
harmony_client = Client(auth=(username.value, password.value))

Next, we create a Collection object with the CMR collection id for our test collection. We then create a Request which specifies the collection, and a `spatial` `BBox` describing the bounding box for the area we're interested in. We'll see later in the notebook how to make sure the request we have is valid.

In [ ]:
collection = Collection(id='C1234088182-EEDTEST')

request = Request(
    collection=collection,
    spatial=BBox(-165, 52, -140, 77)
)

Now that we have a request, we can submit it to Harmony using the Harmony Client object we created earlier. We'll get back a JSON object that describes the Harmony job that we've submitted. As you can see, the Job contains the original request that was submitted to Harmony, its status, as well as a URL where we can get the Job's current status, and its results if it has completed.

In [ ]:
job1 = harmony_client.submit(request)

helper.JSON(job1)

Now using our helper module, we'll wait for the Job to complete, and then we'll download the results and view them.

In [ ]:
helper.download_and_show_results(harmony_client, job1)

Now we show a Harmony request for a temporal range: one month in 2020. As before, we create a Request, and submit it with the same Harmony Client we used above.

In [ ]:
request = Request(
    collection=collection,
    temporal={
        'start': dt.datetime(2020, 6, 1),
        'stop': dt.datetime(2020, 6, 30)
    })

job2 = harmony_client.submit(request)

helper.JSON(job2)

In [ ]:
helper.download_and_show_results(harmony_client, job2)

Finally, we show a Harmony request for both a spatial and temporal range. We create the Request and simply specify both a `spatial` bounds and a `temporal` range, submitting it with the Harmony Client.

In [ ]:
request = Request(
    collection=collection,
    spatial=BBox(-165, 52, -140, 77),
    temporal={
        'start': dt.datetime(2010, 1, 1),
        'stop': dt.datetime(2020, 12, 30)
    })

job3 = harmony_client.submit(request)

helper.JSON(job3)

In [ ]:
helper.download_and_show_results(harmony_client, job3)

Now that we know how to make a request, let's investigate how the Harmony Py library can help us make sure we have a valid request. Recall that we used the Harmony `BBox` type to provide a spatial constraint in our request. If we investigate its help text, we see that we create a `BBox` by providing the western, southern, eastern, and northern latitude/longitude bounds for a bounding box.

In [ ]:
help(BBox)

Now let's create an invalid bounding box by specifying a longitude less than -180 and a northern latitude less than its southern bounds:

In [ ]:
collection = Collection(id='C1234088182-EEDTEST')

request = Request(
    collection=collection,
    spatial=BBox(-183, 40, 10, 30)
)

print(f"Request valid? {request.is_valid()}")
for m in request.error_messages():
    print(" * " + m)

Similarly, we can see errors in the temporal parameter:

In [ ]:
collection = Collection(id='C1234088182-EEDTEST')

request = Request(
    collection=collection,
    temporal={
        'start': dt.datetime(2020, 12, 30),
        'stop': dt.datetime(2010, 1, 1)
    }
)

print(f"Request valid? {request.is_valid()}")
for m in request.error_messages():
    print(" * " + m)

So before submitting a Harmony Request, you can test your request to see if it's valid and how to fix it if not:

In [ ]:
collection = Collection(id='C1234088182-EEDTEST')

request = Request(
    collection=collection,
    spatial=BBox(-183, 40, 10, 30),
    temporal={
        'start': dt.datetime(2020, 12, 30),
        'stop': dt.datetime(2010, 1, 1)
    }
)

print(f"Request valid? {request.is_valid()}")
for m in request.error_messages():
    print(" * " + m)

If we don't validate the request first, Harmony Py will validate it automatically and raise an exception with a message indicating the errors that need to be fixed:

In [ ]:
try:
    harmony_client.submit(request)
except Exception as e:
    print("Harmony Py raised an exception:\n")
    print(e)